# Conditional Generator

This is a demo of the Okareo conditional generator. The conditional generator is useful when you would like to automatically reword questions to emphasize specific conditions of the question. This generator can be used to ensure that minor changes to questions do not adversely affect your LLM/RAG system's retrieval capabilities.

For example, suppose that the following `question` is provided as an input to the generator:

```How can I improve my credit score with no credit history?```

Then the rephrase generator might produce the following:

```Given that I have no credit history, what can I do to improve my credit score?```

```Considering that I do not have any credit history, what steps can I take to improve my credit score?```

Set up a few questions to use in our source scenario for the generator.

In [4]:
data_questions = [
    "What is the name of the file in the current directory?",
    "How do I filter the column of a dataframe based on string length?",
    "Where is the documentation for the Okareo library?"
]
N_data = len(data_questions)

Set up Okareo object with your [API key](https://docs.okareo.com/docs/guides/environment#setting-up-your-okareo-environment).

In [5]:
import os
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetGenerate, SeedData, ScenarioType

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

Add the seed scenario using questions as inputs, return results, and show link to UI.

In [7]:
scenario_set_create = ScenarioSetCreate(name="Conditional Generator: Seed Scenario",
                                        generation_type=ScenarioType.SEED,
                                        number_examples=N_data,
                                        seed_data=[SeedData(input_=data_questions[i], result="example result") for i in range(N_data)]
                                        )

source_scenario = okareo.create_scenario_set(scenario_set_create)

print(source_scenario.app_link)

https://app.okareo.com/project/21b8a05b-f5b6-4578-a36a-fc264036d9d3/scenario/a42d7082-ce56-4996-8d93-7469cc911f55


Use the conditional seed scenario to generate two examples per each point in the seed scenario, and show link to UI.

In [8]:
number_examples = 2 # number of examples to generate for each example in the seed scenario

generated_scenario = okareo.generate_scenario_set(
    ScenarioSetGenerate(
        source_scenario_id=source_scenario.scenario_id,
        name="Conditional Example: Generated Scenario",
        number_examples=number_examples,
        generation_type=ScenarioType.CONDITIONAL
    )
)
print(generated_scenario.app_link)

https://app.okareo.com/project/21b8a05b-f5b6-4578-a36a-fc264036d9d3/scenario/007846de-63fe-4d8c-b0c9-c3cf95f93e85


Compare each generated question to the corresponding source question. 

In [9]:
# compare the descriptions
generated_points = okareo.get_scenario_data_points(generated_scenario.scenario_id)

for i in range(N_data):
    print("-"*8 + f"Example {i}" + "-"*8)
    print(f"Source: {data_questions[i]}")
    for j in range(number_examples):
        print(f"Generated #{j}: {generated_points[number_examples*i+j].input_}")
    

--------Example 0--------
Source: What is the name of the file in the current directory?
Generated #0: Considering the current directory, what would be the name of the file within?
Generated #1: Should you be working in the current directory, what is the name of the file you're dealing with?
--------Example 1--------
Source: How do I filter the column of a dataframe based on string length?
Generated #0: Considering this dataframe, how can one filter the column based on string length?
Generated #1: When dealing with a dataframe, how can I filter a column depending on the length of the string?
--------Example 2--------
Source: Where is the documentation for the Okareo library?
Generated #0: Should you be seeking the documentation for the Okareo library, where might you find it?
Generated #1: In the event that you need the documentation for the Okareo library, where would it be located?
